In [ ]:
import pandas as pd
import numpy as np
import dalex as dx

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [ ]:
train_data = pd.read_csv("./data/train-data.csv", sep=";")

In [ ]:
train_data = train_data.drop_duplicates("index")
train_data

In [ ]:
train_data.columns

In [ ]:
train_data["Month 1"] = train_data["Month 1"].apply(lambda s: "".join(str(s).split())).astype("float")
train_data["Month 2"] = train_data["Month 2"].apply(lambda s: "".join(str(s).split())).astype("float")
train_data["Month 3"] = train_data["Month 3"].apply(lambda s: "".join(str(s).split())).astype("float")
train_data["Month 4"] = train_data["Month 4"].apply(lambda s: "".join(str(s).split())).astype("float")

In [ ]:
train_data

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import FunctionTransformer

X = train_data.drop(columns="Month 4")
y = train_data["Month 4"]

categorical_features    = [
                            "Region",
                            "Country",
                            "Site",
                            "Operations",
                            "Zone",
                            "Cluster",
                            #"Reference proxy",
                            #"Product  Line proxy", #typo !
                            #"Division proxy",
                            "Customer Persona proxy",
                            "Strategic Product Family proxy",
                            "Product Life cycel status", #typo !
                            "Date"
                        ]

numerical_features      = [
                            'Month 1', 'Month 2', 'Month 3'
                        ]

# Pipeline de transformation des features catégoriques 

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist')),
    #('ordinal', OrdinalEncoder())
])

# Pipeline de transformation des features numériques 

numerical_transformer = Pipeline(steps=[
    # ('remove_space', FunctionTransformer(np.frompyfunc(lambda s: "".join(str(s).split()), 1, 1))),
    ('scaler', StandardScaler()),
    ('imputer', SimpleImputer(strategy='constant', fill_value=-1))
])

preprocessor = ColumnTransformer(transformers=[
        ('cat', categorical_transformer, categorical_features),
        #('num', numerical_transformer, numerical_features)
        ('num', numerical_transformer, numerical_features)
])



clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingRegressor(random_state=0))
])

clf.fit(X, y)

In [ ]:
exp = dx.Explainer(clf, X, y)

In [ ]:
exp.model_performance().result